In [ ]:
from pathlib import Path
from itertools import chain
import subprocess 
import os
from collections import defaultdict
import itertools

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from IPython.display import display
import pybedtools
import seaborn as sns

In [ ]:
# results_dir = Path("/mnt/stripe/bio/experiments/aging/loci_of_interest.tables")
# sorted_root = Path("/mnt/stripe/bio/experiments/aging/loci.sorted")
# THREADS_N = 32

results_dir = Path("/Volumes/BigData/bio/experiments/aging/loci_of_interest.tables")
sorted_root = Path("/Volumes/BigData/bio/experiments/aging/loci.sorted")
THREADS_N = 8

results_dir.mkdir(exist_ok=True)

# Cleanup

In [ ]:
#pybedtools.set_tempdir("/tmp")
pybedtools.cleanup()
# !rm -rf {sorted_root}
# !rm -rf {results_dir}

# Known annotations

In [ ]:
# loci_root = Path("/mnt/stripe/bio/raw-data/aging/loci_of_interest")
# golden_peaks_root = Path("/mnt/stripe/bio/experiments/aging/peak_calling")
# zinbra_peaks_root = Path("/mnt/stripe/bio/experiments/configs/Y20O20/peaks")

loci_root = Path("/Volumes/BigData/bio/raw-data/aging/loci_of_interest")
golden_peaks_root = Path("/Volumes/BigData/bio/experiments/aging/peak_calling") # *.*Peak
zinbra_peaks_root = Path("/Volumes/BigData/bio/experiments/configs/Y20O20/peaks") # *.bed

signal_root = Path("/mnt/stripe/bio/experiments/signal")

chromhmm_root = loci_root / "chromhmm"

In [ ]:
!ls {loci_root}

In [ ]:
!ls {zinbra_peaks_root}

In [ ]:
!ls {golden_peaks_root}

## ChromHMM

In [ ]:
chromhmm_paths = list(chromhmm_root.glob('*.bed'))
chromhmm_paths.sort(key=lambda p: int(p.name.split(".")[2].split("_")[0]))

CHROMHMM_ST_MAP = {
    "1_TssA": "Active TSS",
    "2_TssFlnk": "Flanking TSS",
    "3_TssFlnkU": "Flanking TSS Upstream",
    "4_TssFlnkD": "Flanking TSS Downstream",
    "5_Tx": "Strong transcription",
    "6_TxWk": "Weak transcription",
    "7_EnhG1": "Genic enhancer1",
    "8_EnhG2": "Genic enhancer2",
    "9_EnhA1": "Active Enhancer 1",
    "10_EnhA2": "Active Enhancer 2",
    "11_EnhWk": "Weak Enhancer",
    "12_ZNF_Rpts": "ZNF genes & repeats",
    "13_Het": "Heterochromatin",
    "14_TssBiv": "Bivalent/Poised TSS",
    "15_EnhBiv": "Bivalent Enhancer",
    "16_ReprPC": "Repressed PolyComb",
    "17_ReprPCWk": "Weak Repressed PolyComb",
    "18_Quies": "Quiescent/Low",
}

def chromhmm_state_descr(s):
    chunks = s.split(".")
    if len(chunks) > 2:
        state = chunks[2]
        if state in CHROMHMM_ST_MAP:
            return "{} ({})".format(CHROMHMM_ST_MAP.get(chunks[2]), chunks[2])
    return s

for i, p in enumerate(chromhmm_paths):
    print(chromhmm_state_descr(p.name), "->", p)

## Basic Loci

Cannot include all files from dir, because list is too big and heatmap becomes unreadable. Let's keep curated list
of loci by rules:
* root folder top level *.bed files
* subfoldes: "enchancers", "tfs", "regulatory", "weak_consensus", "zinbra_consensus"

In [ ]:
loci_paths = [p for p in loci_root.glob('*.bed')]
for folder in ["enchancers", "tfs", "regulatory", "repeats"]:
    loci_paths.extend([p for p in (loci_root / folder).glob('**/*.bed')])
for folder in ["golden_consensus", "zinbra_consensus"]:
    loci_paths.extend([p for p in (loci_root / folder).glob('*.bed') if ("OD" not in p.name) and ("YD" not in p.name)])
loci_paths = sorted(loci_paths)
loci_paths

## Peaks

In [ ]:
def donor_order_id(path):
    chunks = path.name.split('_')
    cands = list(filter(lambda s: len(s) > 2 and (s.startswith("OD") or s.startswith("YD")), chunks))
    if len(cands) > 0:
        donor_id = cands[0]
        if donor_id[2] != "S":
            return (donor_id[:2], int(donor_id[2:]))

    return (path.name, 0)
    

def collect_peaks(peaks_roots):
    result = {}
    for peaks_root in [x for x in peaks_roots.iterdir() if x.is_dir() and x.name.startswith("H")]:
        print("Peaks:", peaks_root)

        peaks = list(chain(peaks_root.glob("**/*_peaks.bed"),
                           peaks_root.glob("**/bed/*-island.bed"),
                           peaks_root.glob("**/bed/*.*Peak")
                           # peaks_root.glob("**/*consensus*.bed")  # Ignore
                          ))
        for p in peaks:
            assert "outlier" not in str(p)
        # e.g. 
        # * OD_OD14_H3K27ac_hg19_1.0E-6_peaks.bed
        # * OD8_k27ac_hg19_broad_peaks.broadPeak
        # * Ignore consensus: e.g. zinbra_weak_consensus.bed
        peaks.sort(key=donor_order_id)
        print(len(peaks))    
        print(*[str(p) for p in peaks], sep="\n")
        result[peaks_root.name] = peaks
    return result

In [ ]:
golden_peaks_by_histmod = collect_peaks(golden_peaks_root)
golden_peaks_by_histmod.keys()

In [ ]:
zinbra_peaks_by_histmod = collect_peaks(zinbra_peaks_root)
zinbra_peaks_by_histmod.keys()

## Consensus

In [ ]:
zinbra_conensus_paths = [p for p in (loci_root / "zinbra_consensus").glob('*.bed')]
zinbra_conensus_paths

# Alternative:
# zinbra_peaks_by_histmod = collect_peaks(zinbra_peaks_root)
# consensus_peaks = []
# for mod, peaks in zinbra_peaks_by_histmod.items():
#     consensus_peaks.extend([p for p in peaks if "consensus" in p.name])

In [ ]:
golden_conensus_paths = [p for p in (loci_root / "golden_consensus").glob('*.bed')]
golden_conensus_paths

# Alternative:
# golden_peaks_by_histmod = collect_peaks(golden_peaks_root)
# consensus_peaks = []
# for mod, peaks in golden_peaks_by_histmod.items():
#     consensus_peaks.extend([p for p in peaks if "consensus" in p.name])

## Summary

In [ ]:
all_loci = loci_paths + chromhmm_paths

# Code

In [ ]:
!which bedtools

In [ ]:
import sys

# bedtrace.py
def run(commands, stdin=None, stdout=subprocess.PIPE, stderr=subprocess.PIPE):
    """Launches pipe of commands given stdin and final stdout, stderr"""
    processes = []
    _stdin = stdin
    
    try:
        for i, cmd in enumerate(commands):
            if i < len(commands) - 1:
                _stdout = subprocess.PIPE
            else:
                _stdout = stdout

            p = subprocess.Popen(cmd, stdin=_stdin, stdout=_stdout,
                                 stderr=stderr)
            processes.append(p)
            _stdin = p.stdout
    finally:
        for i in range(0, len(processes)):
            try:
                if i < len(processes) - 1:
                    # Allow p1 to receive a SIGPIPE if p2 exits.
                    processes[i].stdout.close()
                else:
                    return processes[i].communicate()
            except Exception:
                exc_type, exc_value, exc_traceback = sys.exc_info()
                print("Error running: {}".format(commands))
                # exc_type below is ignored on 3.5 and later
                traceback.print_exception(exc_type, exc_value, exc_traceback,
                                          #limit=2, 
                                          file=sys.stdout)

In [ ]:
import shutil
def as_sorted(p: Path, root: Path, sorted_root: Path):
    sorted_p = sorted_root / p.relative_to(root)
    sorted_p = sorted_p.parent / (sorted_p.stem + ".sorted.bed")

    if not sorted_p.exists():
        sorted_p.parent.mkdir(exist_ok=True, parents=True)
        
        # Do not resort file if already sorted:
        stderr = run((["sort", "-c", "-k1,1", "-k2,2n", str(p)],))[1]
        is_sorted = (len(stderr) == 0)
        
        if not is_sorted:
            print("Sorting: ", str(p))
            # By some reason BedTool.sort() fails to sort cds.csv
            # bt.sort().saveas(sorted_p)
            #stderr = run((["sort", "-c", "-k1,1", "-k2,2n", str(sorted_p)],))[1]
            #assert len(stderr) == 0, "Expected to be sorted: {}\nError:\n{}".format(sorted_p, stderr)
            with open(str(sorted_p), "w") as f:
                run((["sort", "-k1,1", "-k2,2n", str(p)],), stdout=f)
            print("  [Done]", str(sorted_p))
        else:   
            # just copy file
            shutil.copyfile(str(p), str(sorted_p))
        
        
    return sorted_p

In [ ]:
# def as_sorted_bedtool(p: Path, root: Path, sorted_root: Path):
#     sorted_p = sorted_root / p.relative_to(root)
#     sorted_p = sorted_p.parent / (sorted_p.stem + ".sorted.bed")

#     if not sorted_p.exists():
#         sorted_p.parent.mkdir(exist_ok=True, parents=True)
        
#         # Do not resort file if already sorted:
#         stderr = run((["sort", "-c", "-k1,1", "-k2,2n", str(p)],))[1]
#         is_sorted = (len(stderr) == 0)
        
#         bt = pybedtools.bedtool.BedTool(str(p))
#         if not is_sorted:
#             print("Sorting: ", str(p))
#             # By some reason BedTool.sort() fails to sort cds.csv
#             # bt.sort().saveas(sorted_p)
#             #stderr = run((["sort", "-c", "-k1,1", "-k2,2n", str(sorted_p)],))[1]
#             #assert len(stderr) == 0, "Expected to be sorted: {}\nError:\n{}".format(sorted_p, stderr)
#             with open(str(sorted_p), "w") as f:
#                 run((["sort", "-k1,1", "-k2,2n", str(p)],), stdout=f)
#             print("  [Done]", str(sorted_p))
#         else:   
#             # just copy file
#             bt.saveas(str(sorted_p))
#         del bt  # Too many open files issue
        
#     return pybedtools.bedtool.BedTool(str(sorted_p))

In [ ]:
from multiprocessing import Pool, TimeoutError

def run_bedtools_uniq_wc(ij, a, b):
    output = run((["bedtools", "intersect", "-a", str(a),
                   "-b", str(b), "-wa"],
                  ["uniq"], ["wc", "-l"]))
    return (ij, int(output[0].decode().strip()))

# def run_bedtools_uniq_wc(ij, a: pybedtools.BedTool, b: pybedtools.BedTool):
#     # a = as_sorted_bedtool(a)
#     # b = as_sorted_bedtool(b)
#     c = a.intersect(b, wa=True)
#     output = run((["cat", c.fn], ["uniq"], ["wc", "-l"]))
#     del c  # To many open files issues
#     return (ij, int(output[0].decode().strip()))

def run_bedtools_jaccard(ij, a, b):
    output = run([("bash", "-c", "~/work/washu/bed/jaccard.sh '{}' '{}'".format(str(a), str(b)))])
    stdout = output[0].decode().strip()
    return (ij, float(stdout))

# def run_bedtools_jaccard(ij, a, b):
#     #bed tools jaccard not symmetrix
#     output = run((["bedtools", "jaccard", "-a", str(a),
#                    "-b", str(b)],
#                   ["cut", "-f", "3"]))
#     stdout = output[0].decode().strip()
#     lines = stdout.split("\n")
#     assert len(lines) == 2, lines
#     assert lines[0] == "jaccard"
#     return (ij, float(lines[1]))

# def run_bedtools_jaccard(ij, a: pybedtools.BedTool, b: pybedtools.BedTool):
#     # a = as_sorted_bedtool(a)
#     # b = as_sorted_bedtool(b)
#     return (ij, a.jaccard(b)["jaccard"])

def calc_intersection_table(a_paths, b_paths, path_to_sorted,
                            threads=4, timeout_hours=10, jaccard=False):   
    path_pairs = []
    for i, a in enumerate(a_paths, 0):
        for j, b in enumerate(b_paths, 1):
            path_pairs.append(((i,j), path_to_sorted[a], path_to_sorted[b]))

    metric = run_bedtools_jaccard if jaccard else run_bedtools_uniq_wc
    pool = Pool(processes=threads) 
    multiple_results = [pool.apply_async(metric, 
                                         (ij, a, b)) for ij, a, b in path_pairs]
    values = [res.get(timeout=3600*timeout_hours) for res in multiple_results]
    
    x = np.zeros((len(a_paths), 1 + len(b_paths)), np.float32)
    for (i,j), value in values:
        x[i, j] = value
    
    for i, a in enumerate(a_paths, 0):
        output = run((["cat", str(a)],["wc", "-l"],))
        x[i, 0] = int(output[0].decode().strip())
               
    df = pd.DataFrame(x,
                      index=[f.name for f in a_paths],
                      columns=["total"] + [f.name for f in b_paths])
    return df

In [ ]:
from scipy.cluster import hierarchy
import scipy.spatial as sp

def plot_heatmap(title, df, path=None, autoscale=False, label_fun=None, figsize=(10,10),
                 col_cluster=False, row_cluster=False,
                 as_dist_matrix=False, 
                 row_donors=False):
    if autoscale:
        vmin, vmax = None, None
    else:
        vmin, vmax = 0, 1
        
    if label_fun:
        df = df.copy()
        df.columns = [label_fun(s) for s in df.columns]
        df.index = [label_fun(s) for s in df.index]
  
    if row_donors:
        donors_colors = []
        for fname in df.index:
            chunks = [ch.lower() for ch in fname.split("_")]

            if any(1 for ch in chunks if ch.startswith("od")):
                donors_colors.append("g")
            elif any(1 for ch in chunks if ch.startswith("yd")):
                donors_colors.append("b")
            else:
                donors_colors.append("gray")
#         donors_colors = ["g" if d.lower().startswith("od") else ("b" if d.lower().startswith("YD") else "b")
#                      for d in df.index]
        row_colors = pd.Series(data=donors_colors, index=df.index, name="age")
    else:
        row_colors = None

#     if as_dist_matrix: #impl only for square matrix
#         dissimilarity = sp.distance.squareform(1 - df)
#         linkage = hierarchy.linkage(dissimilarity, method="average")
#         clusters = hierarchy.dendrogram(linkage, no_plot=True)['leaves']

#         cg = sns.clustermap(df, 
#                             row_linkage=linkage, col_linkage=linkage,
#                             col_cluster=col_cluster, row_cluster=row_cluster,
#                             figsize=figsize, cmap="rainbow",
#                             vmin=vmin, vmax=vmax
#                            )
#         plt.setp(cg.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
#         plt.setp(cg.ax_heatmap.xaxis.get_majorticklabels(), rotation=90)
#     else:
    g = sns.clustermap(df,
                       col_cluster=col_cluster, row_cluster=row_cluster,
                       figsize=figsize, cmap="rainbow",
                       metric="chebyshev",
                       vmin=vmin, vmax=vmax, robust=True, #robust=True: ignore color outliers
                       row_colors=row_colors)
    
    plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)
    
    plt.title(title)
    if path is None:        
        plt.show()
    else:
        pp.savefig()

In [ ]:
def load_intersection_table(beds, loci, path_to_bt, result_path, threads=4, jaccard=False):
    if result_path.exists():
        df = pd.DataFrame.from_csv(result_path)
        print("Loaded: ", result_path)
    else:
        print("Calculating: ", result_path)
        df = calc_intersection_table(beds, loci, path_to_bt, threads=threads, jaccard=jaccard) 
        result_path.parent.mkdir(parents=True, exist_ok=True)
        df.to_csv(str(result_path))
        print("  Saved: ", result_path)
        
    return df

In [ ]:
def normalize(df):
    return df.divide(df["total"], axis=0).drop("total", axis=1)

In [ ]:
def process_intersection(beds, loci, path_to_bt, results_dir, tag,
                         figsize=(10,10), col_cluster=False, row_cluster=True,
                         all_metrics=0,
                         **kw):
    dfs_n = []
    df_bl = load_intersection_table(beds, loci, path_to_bt, 
                                    results_dir / "{}_bl.csv".format(tag), threads=THREADS_N)
    display(df_bl.head(3))
    
    df_n_bl = normalize(df_bl)
    display(df_n_bl.head(3))
    dfs_n.append(df_n_bl)

    if all_metrics > 0:
        df_lb = load_intersection_table(loci, beds, path_to_bt,
                                        results_dir / "{}_lb.csv".format(tag), threads=THREADS_N)
        display(df_lb.head(3))

        df_n_lb = normalize(df_lb).T
        display(df_n_lb.head(3))
        dfs_n.append(df_n_lb)

    if all_metrics > 2:
        df_jaccard = load_intersection_table(beds, loci, path_to_bt, 
                                             results_dir / "{}_js.csv".format(tag), threads=THREADS_N,
                                             jaccard = True)
        df_jaccard = df_jaccard.drop("total", axis=1)
        display(df_jaccard.head(3))
        dfs_n.append(df_jaccard)
    
    def inner_chromhmm_or_donor(name):
        if "row_donors" in kw:
            chunks = name.split('_')
            cands = list(filter(
                lambda s: len(s) > 2 and (s.startswith("OD") or s.startswith("YD")) and s[2] != "S",
                chunks))
            
            if len(cands) > 0:
                return cands[0]
            
        return chromhmm_state_descr(name)
    
    plot_heatmap("Metrics: # intervals from row file intersecting any interval from column file",
                 df_n_bl, autoscale=False, label_fun=inner_chromhmm_or_donor, figsize=figsize,
                 col_cluster=col_cluster, row_cluster=row_cluster, **kw)
    
    if all_metrics > 0:
        plot_heatmap("Metrics: # intervals from col file intersecting any interval from row file",
                     df_n_lb, autoscale=False, label_fun=inner_chromhmm_or_donor, figsize=figsize,
                     col_cluster=col_cluster, row_cluster=row_cluster, **kw)
    
    if all_metrics > 1:
        plot_heatmap("Metrics: Geometric mean for intersectiong intervals",
                     np.sqrt(df_n_bl*df_n_lb), autoscale=False, label_fun=inner_chromhmm_or_donor, figsize=figsize,
                     col_cluster=col_cluster, row_cluster=row_cluster, **kw)

    if all_metrics > 2:
        plot_heatmap("Metrics: Jaccard",
                     df_jaccard, autoscale=True, label_fun=inner_chromhmm_or_donor, figsize=figsize,
                     col_cluster=col_cluster, row_cluster=row_cluster,
                     as_dist_matrix=True, **kw)
    return dfs_n

In [ ]:
def check_interests_loci_folder(folder_name,):
    paths = sorted([p for p in (loci_root / folder_name).glob('**/*.bed')])
    suffix = folder_name
    print("Paths: #", len(paths))

    # ChromHMM
    print("Ensure files sorted...")
    mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in paths}
    mapping.update({p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in chromhmm_paths})
    print("[Done]")
    process_intersection(paths, chromhmm_paths, mapping, results_dir, "{}_chromhmm".format(suffix), figsize=(8,8),
                         all_metrics=1)
    
    # Consensus: Y+O
    consensus_paths = sorted([p for p in (zinbra_conensus_paths + golden_conensus_paths) if "DS" not in p.name],
                             key=lambda p: p.name)
    print("Ensure files sorted...")
    mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in paths}
    mapping.update({p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in consensus_paths})
    print("[Done]")
    process_intersection(paths, consensus_paths, mapping, results_dir, "{}_consensus_short".format(suffix),
                         figsize=(8,8), row_cluster=True, col_cluster=False,
                         all_metrics=1)

    # Consensus: Y,O,Y+O
    consensus_paths = sorted(zinbra_conensus_paths + golden_conensus_paths,
                             key=lambda p: p.name)
    print("Ensure files sorted...")
    mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in paths}
    mapping.update({p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in consensus_paths})
    print("[Done]")
    process_intersection(paths, consensus_paths, mapping, results_dir, "{}_consensus".format(suffix),
                         figsize=(12,8), row_cluster=True, col_cluster=False)

    # Loci
    print("Ensure files sorted...")
    mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in paths}
    mapping.update({p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in loci_paths})
    print("[Done]")
    process_intersection(paths, loci_paths, mapping, results_dir, "{}_loci".format(suffix), figsize=(20,8),
                         all_metrics=1)

# TMP

In [ ]:
!rm {results_dir}/tmp0_loci.csv*

In [ ]:
print("Ensure files sorted...")
tmp_loci_paths = loci_paths[0:6]
mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in tmp_loci_paths}
print("[Done]")

process_intersection(tmp_loci_paths, tmp_loci_paths, mapping, results_dir, "tmp0_loci.csv", figsize=(5,5))
None

# Chose Metrics

In [ ]:
print("Ensure files sorted...")
mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in loci_paths}
print("[Done]")

process_intersection(loci_paths, loci_paths, mapping, results_dir, "loci", figsize=(15,15),
                     col_cluster=True,
                     all_metrics=True)
None

# Peaks

## Loci vs Loci

In [ ]:
print("Ensure files sorted...")
mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in loci_paths}
print("[Done]")

process_intersection(loci_paths, loci_paths, mapping, results_dir, "loci", figsize=(15,15),
                     col_cluster=True)
None

## Loci vs ChromHMM

In [ ]:
print("Ensure files sorted...")
mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in loci_paths}
mapping.update({p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in chromhmm_paths})
print("[Done]")

process_intersection(loci_paths, chromhmm_paths, mapping, results_dir, "loci_chromhmm", figsize=(8, 15))
None

## ChromHMM vs ChromHMM

In [ ]:
print("Ensure files sorted...")
mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in chromhmm_paths}
print("[Done]")
process_intersection(chromhmm_paths, chromhmm_paths, mapping, results_dir, "chromhmm", figsize=(8,8))
None

## Consensus vs Consensus

In [ ]:
print("Ensure files sorted...")
mapping = {}
consensus_paths = sorted([p for p in (zinbra_conensus_paths + golden_conensus_paths) if "DS" not in p.name],
                         key=lambda p: p.name)
for p in (consensus_paths):
    mapping[p] = as_sorted(p, loci_root, sorted_root / "loci_of_interest")
print("[Done]")

dfs = process_intersection(consensus_paths, consensus_paths, mapping, results_dir, "consensus_short", figsize=(6,6),
                     col_cluster=True)
dfs[0]

In [ ]:
print("Ensure files sorted...")
mapping = {}
consensus_paths = sorted(zinbra_conensus_paths + golden_conensus_paths,
                         key=lambda p: p.name)
for p in (consensus_paths):
    mapping[p] = as_sorted(p, loci_root, sorted_root / "loci_of_interest")
print("[Done]")

process_intersection(consensus_paths, consensus_paths, mapping, results_dir, "consensus", figsize=(15,8),
                     col_cluster=False)
None

## Zinbra vs Loci

### Consensus peaks

In [ ]:
print("Ensure files sorted...")
mapping = {}
for p in zinbra_conensus_paths:
    mapping[p] = as_sorted(p, loci_root, sorted_root / "loci_of_interest")
for p in all_loci:
    mapping[p] = as_sorted(p, loci_root, sorted_root / "loci_of_interest")
print("[Done]")

process_intersection(zinbra_conensus_paths, all_loci, mapping, results_dir, "zinbra_consensus_vs_loci", 
                     figsize=(20,4), row_donors=True)
None

### All Hist mods:

In [ ]:
print("Ensure files sorted...")
mapping = {}
for mod, peaks in zinbra_peaks_by_histmod.items():
    for p in peaks:
        mapping[p] = as_sorted(p, zinbra_peaks_root, sorted_root / "zinbra")
for p in all_loci:
    mapping[p] = as_sorted(p, loci_root, sorted_root / "loci_of_interest")
print("[Done]")

for mod, peaks in zinbra_peaks_by_histmod.items():
    print(mod)
    process_intersection(peaks, all_loci, mapping, results_dir, "zinbra_{}_vs_loci".format(mod),
                         figsize=(20,10), col_cluster=True, row_cluster=True)
None    

## Macs vs Loci

### Conensus peaks:

In [ ]:
print("Ensure files sorted...")
mapping = {}
for p in golden_conensus_paths:
    mapping[p] = as_sorted(p, loci_root, sorted_root  / "loci_of_interest")
for p in all_loci:
    mapping[p] = as_sorted(p, loci_root, sorted_root / "loci_of_interest")
print("[Done]")

process_intersection(golden_conensus_paths, all_loci, mapping, results_dir, "golden_consensus_vs_loci",
                     figsize=(20,4), row_donors=True)
None

### All Hist mods:

In [ ]:
print("Ensure files sorted...")
mapping = {}
for mod, peaks in golden_peaks_by_histmod.items():
    for p in peaks:
        mapping[p] = as_sorted(p, golden_peaks_root, sorted_root / "golden")
for p in all_loci:
    mapping[p] = as_sorted(p, loci_root, sorted_root / "loci_of_interest")
print("[Done]")

for mod, peaks in golden_peaks_by_histmod.items():
    print(mod)
    process_intersection(peaks, all_loci, mapping, results_dir, "golden_{}_vs_loci".format(mod),
                         figsize=(20,10), row_donors=True,
                         col_cluster=True, row_cluster=True)
None

## Pathways

In [ ]:
# check_interests_loci_folder("pathway")
# to many data to calc..

## Repeats

In [ ]:
check_interests_loci_folder("repeats")
None

# Diff Peaks

##  ChipSeq Diff

### vc ChromHmm, Loci, Consensus

In [ ]:
check_interests_loci_folder("chipseq_diff_loci")
None

In [ ]:
###  vs ChromHMM

# diff_chip_root = loci_root / "chipseq_diff_loci"
# diff_chip_paths = [p for p in diff_chip_root.glob('**/*.bed')]

# print("Ensure files sorted...")
# mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in diff_chip_paths}
# mapping.update({p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in chromhmm_paths})
# print("[Done]")
# process_intersection(diff_chip_paths, chromhmm_paths, mapping, results_dir, "diff_chip_chromhmm", figsize=(8,8))

### vs Loci
# diff_chip_root = loci_root / "chipseq_diff_loci"
# diff_chip_paths = [p for p in diff_chip_root.glob('**/*.bed')]

# print("Ensure files sorted...")
# mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in diff_chip_paths}
# mapping.update({p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in loci_paths})
# print("[Done]")
# process_intersection(diff_chip_paths, loci_paths, mapping, results_dir, "diff_chip_loci", figsize=(8,8))


### vs Consensus
# diff_chip_root = loci_root / "chipseq_diff_loci"
# diff_chip_paths = [p for p in diff_chip_root.glob('**/*.bed')]

# consensus_paths = zinbra_conensus_paths + golden_conensus_paths
# print("Ensure files sorted...")
# mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in diff_chip_paths}
# mapping.update({p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in consensus_paths})
# print("[Done]")
# process_intersection(diff_chip_paths, consensus_paths, mapping, results_dir, "diff_chip_consensus",
#                      figsize=(8,8), row_cluster=False, col_cluster=True)


### vs Repeats

In [ ]:
diff_chip_paths = [p for p in (loci_root / "chipseq_diff_loci").glob('**/*.bed')]
repeats_paths = [p for p in (loci_root / "repeats").glob('**/*.bed')]

print("Ensure files sorted...")
mapping = {p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in diff_chip_paths}
mapping.update({p:as_sorted(p, loci_root, sorted_root / "loci_of_interest") for p in repeats_paths})
print("[Done]")
process_intersection(diff_chip_paths, repeats_paths, mapping, results_dir, "chipseq_diff_repeats",
                     figsize=(8,8), row_cluster=False, col_cluster=True)
None

## RnaSeq diff

In [ ]:
check_interests_loci_folder("rna_diff")
None

# Coverage (Signal)

In [ ]:
signal_root

In [ ]:
signals_results_dir = results_dir / "signals"

## Plots

In [ ]:
! rm -rf {signals_results_dir}

In [ ]:
if signals_results_dir.exists():
    # TODO: load
    pass
else:
    signal_dfs_by_datatype = {}
    signal_dfs_by_loci = {}
    missed_files = []

    signals_results_dir.mkdir(exist_ok=True, parents=True)
    
    series_by_loci = defaultdict(list)
    data_type_paths = [p for p in signal_root.iterdir() if p.is_dir()]
    for i, data_type_path in enumerate(data_type_paths, 1):
        data_type = data_type_path.name
        print("[{}/{}] Processing: {}".format(i, len(data_type_paths), data_type))

        for norm in ["raw", "rpkm", "rpm"]:
            print("  Normalization:", norm)
            series_by_datatype = []

            # TODO: load from results dir?
            for loci_path in (p for p in data_type_path.iterdir() if p.is_dir()):
                loci = loci_path.name
                files = [p for p in loci_path.glob("**/*_{}_data.csv".format(norm))]

                assert len(files) <= 1, "{}@{} [{}] Expected one file, but was {}: {}".format(
                    data_type, loci, norm, len(files), files
                )
                if not len(files):
                    missed_files.append("{}@{} [{}]".format(data_type, loci, norm))
                    continue

                df = pd.DataFrame.from_csv(files[0] , header=None)
                series = df.iloc[:,0]
                series.name = loci

                series_by_datatype.append(series) 

                series2 = series.copy()
                series2.name = data_type
                series_by_loci[(loci, norm)].append(series2)

            if len(series_by_datatype):
                # by data type:    
                df = pd.DataFrame(series_by_datatype, )
                #df.index = [f.stem for f in itertools.islice(files, 10)]

                df.to_csv(str(signals_results_dir / "signal_{}_{}.csv".format(data_type, norm)))
            else:
                df = None
            signal_dfs_by_datatype[(data_type, norm)] = df

    for (loci, norm), series in series_by_loci.items():
        if len(series):
            df = pd.DataFrame(series, )
            df.to_csv(str(signals_results_dir / "signal_{}_{}".format(loci, norm)))
        else:
            df = None
        signal_dfs_by_loci[(loci, norm)] = df
        
    print("Missed files: ", len(missed_files))
    print("  first 10:", *missed_files[0:10])
    print("Signal by datatype, missed records:", str(sum(1 for v in signal_dfs_by_datatype.values() if v is None)))
    print("  ", [k for k,v in signal_dfs_by_datatype.items() if v is None])
    print("Signal by loci, missed records:", str(sum(1 for v in signal_dfs_by_loci.values() if v is None)))    
    print("  ", [k for k,v in signal_dfs_by_loci.items() if v is None])

In [ ]:
signal_dfs_by_datatype[("H3K4me1", "rpkm")].head()

In [ ]:
signal_dfs_by_loci[("washu_german_rrbs_filtered_dmrs_all_10.hg19", "rpkm")].head()

In [ ]:
def plot_donors_heatmap(title, df, path=None, autoscale=False, 
                        label_fun=None, figsize=(10,10),
                        col_cluster=False, row_cluster=False):
    if autoscale:
        vmin, vmax = None, None
    else:
        vmin, vmax = 0, 1
        
    if label_fun:
        df = df.copy()
        df.columns = [label_fun(s) for s in df.columns]
        df.index = [label_fun(s) for s in df.index]
        
    donors_colors = ["g" if d.lower().startswith("od") else ("b" if d.lower().startswith("YD") else "b")
                     for d in df.index]
    row_colors = pd.Series(data=donors_colors, index=df.index, name="age")
            
    g = sns.clustermap(df,
                       col_cluster=col_cluster, row_cluster=row_cluster,
                       figsize=figsize, cmap="rainbow",
                       metric="chebyshev",
                       standard_scale = None,
                       vmin=vmin, vmax=vmax,
                       row_colors=row_colors,
                       robust=True) #robust=True: ignore color outliers
    plt.setp(g.ax_heatmap.get_yticklabels(), rotation=0)

    plt.title(title)
    if path is None:        
        plt.show()
    else:
        pp.savefig()
        
def plot_signal_heatmap(tag, metric, signal_dfs, *args,
                        col_filter_fun=None,
                        **kw):

    df = signal_dfs[(tag, metric)]
    if df is None:
        return

    df = df.T
        
    # let's sort by index, not just lexicographically, but in human readable order, e.g. OD2 shoud be before OD10
    def inner_donor_order_id(name):
        assert (len(name) > 2 and (name.startswith("od") or name.startswith("yd")))
        return (name[:2], int(name[2:]))

    df = df.loc[sorted(df.index.tolist(), key=inner_donor_order_id), :]
    
    if col_filter_fun:
        df = df.loc[:, [c for c in df.columns if col_filter_fun(c)]]

    # Normalize by columns (by loci across all donors)
    df = ((df - np.min(df, axis=0))/(np.max(df, axis=0) - np.min(df, axis=0)))
    plot_donors_heatmap("[{}]: {}".format(metric, tag), df, *args, **kw)

In [ ]:
# Not pathways:{k for k,v in signal_dfs_by_loci.keys() if not k.startswith("R")}

### All signal @ CGI

In [ ]:
for norm in ["raw", "rpkm", "rpm"]:
    plot_signal_heatmap("ucsc_cpgIslandExt.hg19", norm, signal_dfs_by_loci, 
                        #col_filter_fun=lambda x: x == "meth",
                        row_cluster=False, col_cluster=False)

### All signal @ (DMR, 14_TssBiv, 15_Enh_Biv)

In [ ]:
for loci in ['cd14_chromhmm.hg19.14_TssBiv', 'cd14_chromhmm.hg19.15_EnhBiv', "washu_german_rrbs_filtered_dmrs_all_10.hg19"]:
    for norm in ["raw", "rpkm", "rpm"]:
        plot_signal_heatmap(loci, norm, signal_dfs_by_loci, 
                            col_filter_fun=lambda x: x == "H3K4me1",
                            row_cluster=True, col_cluster=False)

### H3K3me1 signal @ loci

In [ ]:
for norm in ["raw", "rpkm", "rpm"]:
    plot_signal_heatmap("H3K4me1", norm, signal_dfs_by_datatype, 
                        col_filter_fun=lambda loci: not loci.startswith("R-HSA"),
                        row_cluster=False, col_cluster=True,
                        figsize=(24, 10))

### Every data type @ ChromHMM

In [ ]:
for norm in ["raw", "rpkm", "rpm"]:
    for histmod in {k for k,v in signal_dfs_by_datatype.keys()}:
        display(norm + ":" + histmod)
        plot_signal_heatmap(histmod, norm, signal_dfs_by_datatype, 
                            col_filter_fun=lambda loci: loci.startswith("cd14_chromhmm"),
                            row_cluster=False, col_cluster=False)

## Stat testing

In [ ]:
signal_root

In [ ]:
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

signal_pvalues = defaultdict(list)
missed_files = []
problem_files = []
ha = "two-sided" # 'less', 'two-sided', or 'greater'
data_type_paths = [p for p in signal_root.iterdir() if p.is_dir()]
for i, data_type_path in enumerate(data_type_paths, 1):
    data_type = data_type_path.name
    print("\n[{}/{}] Processing: {}".format(i, len(data_type_paths), data_type))
    
    for j, loci_path in enumerate(p for p in data_type_path.iterdir() if p.is_dir()):
        loci = loci_path.name
        print(".", end="")

        pvalues = {}
        signal_normalizations = ["raw", "rpkm", "rpm"]
        for norm in signal_normalizations:
            files = [p for p in loci_path.glob("**/*_{}_data.csv".format(norm))]
            
            assert len(files) <= 1, "{}@{} [{}] Expected one file, but was {}: {}".format(
                data_type, loci, norm, len(files), files
            )
            if not len(files):
                missed_files.append("{}@{} [{}]".format(data_type, loci, norm))
                continue
            
            file = files[0]
            df = pd.DataFrame.from_csv(file , header=None)
            df_ods = df.loc[[d for d in df.index if d.startswith("o")],:]
            df_yds = df.loc[[d for d in df.index if d.startswith("y")],:]
            try:
                pvalue = mannwhitneyu(df_ods.iloc[:,0], df_yds.iloc[:,0],
                                      alternative=ha).pvalue
            except ValueError as e:
                print("Error: {} in file:\n{}".format(e, file))
                problem_files.append(file)    
            pvalues[norm] = pvalue

        signal_pvalues["name"].append("{}@{}".format(data_type, loci))    
        for norm in signal_normalizations:
            signal_pvalues[norm].append(pvalues.get(norm, np.nan)) #1.0

print("Missed files: ", len(missed_files))
print("  first 10:", *missed_files[0:10])
print("Errors occurred in files: ", len(problem_files))
print("  first 10:", *problem_files[0:10])

signal_pvalues_df = pd.DataFrame.from_dict(signal_pvalues)
signal_pvalues_df.to_csv(str(results_dir / "signal_pvalues.csv"))
signal_pvalues_df.head(10)

In [ ]:
signal_pvalues_df.index = signal_pvalues_df.name
signal_pvalues_df.drop("name", inplace=True, axis=1)

print("Not corrected pval, first 10 lowerest pvalues:")
signal_pvalues_df["min"] = signal_pvalues_df.min(axis=1)
signal_pvalues_df.sort_values(by="min").head(10)

In [ ]:
def manhattan_plot(pvalues_df, correction="Uncorrected"):
    plt.figure(figsize=(10,10))
    for i, norm in enumerate(["raw", "rpkm", "rpm"], 1):
        n = pvalues_df.shape[0]
        ax = plt.subplot(2, 2, i)
        #plt.plot(range(n), -np.log10(pvalues_df["raw"]), marker=".", ls="")
        #ax.hlines(y=-np.log10(0.05), xmin=0, xmax=n, color="r", linestyle='dotted')
        ax.plot(range(n), 1/pvalues_df[norm], marker=".", ls="")
        ax.axhline(y=-np.log10(0.05), xmin=0, xmax=n, color="r", linestyle='dotted')
        ax.set_ylabel("{} pvalues (-log(p) scale )".format(correction))
        ax.set_yscale("log")
        ax.set_title("[{}] Mann whitney u test pvalues".format(norm))
    plt.show()

In [ ]:
manhattan_plot(signal_pvalues_df)

In [ ]:
# P-values correction
#
# see: http://www.statsmodels.org/dev/_modules/statsmodels/stats/multitest.html
signal_pvalues_bh_df = signal_pvalues_df.copy().drop("min", axis=1)
for c in signal_pvalues_bh_df.columns:
    pvals = signal_pvalues_bh_df.loc[:, c]
    pvals_not_nan_mask = ~np.isnan(pvals)
    pvals_not_nan = pvals[pvals_not_nan_mask]
    _reject, pvals_corrected, *_ = multipletests(pvals=pvals_not_nan, 
                                                 alpha=0.05, method="fdr_bh") #fdr_bh, holm-sidak, bonferroni 
    signal_pvalues_bh_df.loc[pvals_not_nan_mask, c] = pvals_corrected
#     _reject, pvals_corrected, *_ = multipletests(pvals=signal_pvalues_df.loc[:, c], 
#                                                  alpha=0.05, method="fdr_bh")
#     df_fdr_bh[c] = pvals_corrected
    
signal_pvalues_bh_df["min"] = signal_pvalues_bh_df.min(axis=1, skipna=True)
signal_pvalues_bh_sorted_df = signal_pvalues_bh_df.sort_values(by="min")

In [ ]:
manhattan_plot(signal_pvalues_bh_sorted_df, "Benjamini–Hochberg corrected")

In [ ]:
signal_pvalues_bh_sorted_df_005 = signal_pvalues_bh_sorted_df[signal_pvalues_bh_sorted_df["min"] < 0.05]
print("Passing FDR 0.05 by any metric:", len(signal_pvalues_bh_sorted_df_005))
signal_pvalues_bh_sorted_df_005.head(10)

In [ ]:
signal_pvalues_df.loc[signal_pvalues_bh_sorted_df_005.head(10).index, :]

In [ ]:
print("Corrected, first 10 lowerest pvalues:")
display(signal_pvalues_bh_sorted_df.head(10))

print("Same records, but original pvalues:")
display(signal_pvalues_df.loc[signal_pvalues_bh_sorted_df.head(10).index, :])

In [ ]:
np.min(signal_pvalues_bh_sorted_df["raw"]), np.max(signal_pvalues_bh_sorted_df["raw"])

In [ ]:
print("Corrected, dmr related pvalues:")
signal_pvalues_bh_sorted_df.loc[signal_pvalues_bh_sorted_df.index.str.contains("dmr"), :]